<a href="https://colab.research.google.com/github/changdaeoh/DACON_Dirty-MNIST/blob/main/0301_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글드라이브와 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setting

In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

# common modules
import tensorflow as tf
from tensorflow import  keras

import numpy as np
import pandas as pd 
import random

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
os.chdir('/content/drive/MyDrive/project_dataset/dacon_v2')

# 데이터 경로
train_dir = "/content/drive/MyDrive/project_dataset/dacon_v2/dirty_mnist_2nd"
test_dir = "/content/drive/MyDrive/project_dataset/dacon_v2/test_route"

# GPU 확인
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# global seed 고정
SEED = 301

def seed_everything(seed = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

     |████████████████████████████████| 706kB 5.7MB/s 
Found GPU at: /device:GPU:0


# Data Preparing

In [ ]:
test_submit = pd.read_csv('sample_submission.csv')
test_df = test_submit.copy()
test_df['index'] = test_submit['index'].apply(lambda x: str("{:0>5d}".format(x))+'.png')

batch_size = 250


test_gen = ImageDataGenerator(rescale=1./255.,
                              rotation_range = 10,
                              width_shift_range = 0.1,
                              height_shift_range = 0.1,
                              horizontal_flip = True,
                              vertical_flip = True,
                              fill_mode = "nearest")

test_gen = test_gen.flow_from_dataframe(dataframe = test_df,        
                                        directory='./test_route/test_dirty_mnist_2nd',      
                                        x_col='index',                             
                                        batch_size = batch_size,               
                                        shuffle = False,
                                        color_mode = "rgb",           
                                        class_mode=None,
                                        target_size=(256, 256))

Found 5000 validated image filenames.


# TTA

In [ ]:
batch_size = 32
tta_steps = 30
predictions = []
columns = list(test_df.columns[1:])
filename = "incepRes_TTA.csv"

for i in range(tta_steps):
    print("iter {}".format(i))
    preds = model.predict_generator(generator = test_gen, steps = 5000 // batch_size, verbose = 1)
    predictions.append(preds)

# 평균을 통한 final prediction
pred = np.mean(predictions, axis=0)
pred_sub = pred.copy()

# get label
pred_sub = pred_sub.round()

# create DF
res = pd.DataFrame(pred_sub, columns = columns )
int_rest = res.astype(int)
submit = pd.concat([test_submit.iloc[:,0],int_rest], axis = 1)

submit.to_csv(filename, index = False)

# MC Dropout

In [ ]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

mc_model = keras.models.Sequential([
    MCDropout(layer.rate) if isinstance(layer, keras.layers.Dropout) else layer
    for layer in model.layers
])

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
mc_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
mc_model.set_weights(model.get_weights())

np.round(np.mean([mc_model.predict(X_test_scaled[:1]) for sample in range(100)], axis=0), 2)

# TTA + MC Dropout

In [ ]:
seed_everything(SEED)

# load_model
model = keras.models.load_model("./model/effi_0301.h5")

# replace normal DO -> MC DO that forced to have training status
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

mc_model = keras.models.Sequential([
    MCDropout(layer.rate) if isinstance(layer, keras.layers.Dropout) else layer
    for layer in model.layers
])


# LR schedule
n_epochs = 60
first_decay_steps = ((45000 // batch_size) * n_epochs) // 5
initial_learning_rate = 0.003

lr_decayed_fn = (
  tf.keras.experimental.CosineDecayRestarts(
      initial_learning_rate,
      first_decay_steps,
      t_mul=2.0,
      m_mul=0.95))


# optimizer
radam = tfa.optimizers.RectifiedAdam(learning_rate = lr_decayed_fn,
                                     weight_decay = 0.0001,
                                     warmup_proportion = 0,
                                     min_lr = 1e-6)
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)


# compile
BAcc = keras.metrics.BinaryAccuracy(name='binary_accuracy')
mc_model.compile(optimizer = ranger, loss = "binary_crossentropy", metrics = [BAcc])
mc_model.set_weights(model.get_weights())


# Setting for TTA and MCDO
batch_size = 250
tta_steps = 5
mc_steps = 5
predictions = []
columns = list(test_df.columns[1:])
filename = "efficientB0_0301.csv"


# double loop 
for i in range(tta_steps):
    print("*********** augment iter {} ***********".format(i))
    mc_pred = [mc_model.predict_generator(generator = test_gen, 
                                          steps = 5000 // 250,
                                          verbose = 1) for sample in range(mc_steps)]
    mc_pred_means = np.mean(mc_pred, axis = 0)
    predictions.append(mc_pred_means)


# final prediction
pred = np.mean(predictions, axis=0)
pred_sub = pred.copy()

# get label
pred_sub = pred_sub.round()

# create DF
res = pd.DataFrame(pred_sub, columns = columns )
int_rest = res.astype(int)
submit = pd.concat([test_submit.iloc[:,0],int_rest], axis = 1)
submit.to_csv(filename, index = False)

*********** augment iter 0 ***********


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


20/20 [==============================] - 76s 4s/step
*********** augment iter 1 ***********
 1/20 [>.............................] - ETA: 1:15

KeyboardInterrupt: ignored

In [ ]:
# final prediction
pred = np.mean(predictions, axis=0)
pred_sub = pred.copy()

# get label
pred_sub = pred_sub.round()

# create DF
res = pd.DataFrame(pred_sub, columns = columns )
int_rest = res.astype(int)
submit = pd.concat([test_submit.iloc[:,0],int_rest], axis = 1)
submit.to_csv(filename, index = False)